In [257]:
import time
import requests
import urllib.request
from bs4 import BeautifulSoup
import json

def souper(url):
    user_agent = 'Thijmen Kupers Student reseaching arguments on CMV: t.kupers@student.rug.nl'
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html,'html.parser')
    return(soup)

def linksAndTopics(url):
    main_table = souper(url).find("div",attrs={'id':'siteTable'})
    comment_a_tags = main_table.find_all('a',attrs={'class':'bylink comments may-blank'})
    #Blank list to store the urls as they are extracted
    urls = [] 
    for a_tag in comment_a_tags:
        url = a_tag['href']
        if not url.startswith('http'):
            url = "https://reddit.com"+url
        urls.append(url)

    titles = main_table.find_all('a',attrs={'class':'title'})
    topics= []
    for topic in titles:
        t = topic.contents[0]
        topics.append(t)
    return(topics, urls)

url = "https://old.reddit.com/r/changemyview/"


##two vectors of topics and urls
topics, urls = linksAndTopics(url)

In [258]:
print(topics[0])
urls[0]

CMV: Dragonflys are significantly doper than spiders


'https://old.reddit.com/r/changemyview/comments/9ody9p/cmv_dragonflys_are_significantly_doper_than/'

In [259]:


def getOPtext(url):
    main_table = souper(url).find("div",attrs={'id':'siteTable'})
    OPtext = main_table.find("div", attrs={'class':'usertext-body'}).findAll('p')
    txt = ""
    for p in range(len(OPtext)):
        txt += OPtext[p].text

    OPwords = txt.split()


    OPname = main_table.find("a", attrs={'class':"author"})
    name = OPname.text
    
    return(txt, OPwords, name)

txt, OPwords, OPname = getOPtext(urls[0])

#print(newURL + "\n")
#print(topics[0]+ "\n")
#print(txt + "\n")
#print(name)

In [260]:
def getComments(url):
    comment_area = souper(url).find('div',attrs={'class':'commentarea'})
    comments = comment_area.find_all('div', attrs={'class':'entry unvoted'})
    extracted_comments = []
    textComm = []
    names = []
    for comment in comments: 
        if comment.find('form'):
            commenter = comment.find('a',attrs={'class':'author'}).text
            comment_text = comment.find('div',attrs={'class':'md'}).text
            #link = comment.find('a',attrs={'class':'bylink'})['href']
            extracted_comments.append({'name':commenter,'text':comment_text})
            textComm.append(comment_text)
            names.append(commenter)
            
    return(extracted_comments, textComm, names)

extracted_comments, textComm, names = getComments(urls[0])
print(len(extracted_comments))

print(textComm[0].split())
##get individual commments and split it into words
def bagOfWords(textComm):
    words = []
    for i in range(1,len(textComm)):
        words.append(textComm[i].split)
    return(words)

words = bagOfWords(textComm)

144
['/u/mostlikelynotarobot', '(OP)', 'has', 'awarded', '6', 'delta(s)', 'in', 'this', 'post.', 'All', 'comments', 'that', 'earned', 'deltas', '(from', 'OP', 'or', 'other', 'users)', 'are', 'listed', 'here,', 'in', '/r/DeltaLog.', 'Please', 'note', 'that', 'a', 'change', 'of', 'view', "doesn't", 'necessarily', 'mean', 'a', 'reversal,', 'or', 'that', 'the', 'conversation', 'has', 'ended.', 'Delta', 'System', 'Explained', '|', 'Deltaboards']


In [261]:
## all functions together:
## soup = souper(url)
## topics, urls = linksAndTopics(url)
## txt, OPwords, name = getOPtext(url)
## extracted_comments, textComm, names = getComments(url)
## words = bagOfWords(textComm)

In [262]:
names[0]

'DeltaBot'

In [263]:
def souper(url):
    user_agent = 'Thijmen Kupers Student reseaching arguments on CMV: t.kupers@student.rug.nl'
    request = urllib.request.Request(url,headers={'User-Agent': user_agent})
    html = urllib.request.urlopen(request).read()
    soup = BeautifulSoup(html,'html.parser')
    return(soup)

def linksAndTopics(url):
    main_table = souper(url).find("div",attrs={'id':'siteTable'})
    comment_a_tags = main_table.find_all('a',attrs={'class':'bylink comments may-blank'})
    #Blank list to store the urls as they are extracted
    urls = [] 
    for a_tag in comment_a_tags:
        url = a_tag['href']
        if not url.startswith('http'):
            url = "https://reddit.com"+url
        urls.append(url)

    titles = main_table.find_all('a',attrs={'class':'title'})
    topics= []
    for topic in titles:
        t = topic.contents[0]
        topics.append(t)
    return(topics, urls)

def getOPtext(url):
    main_table = souper(url).find("div",attrs={'id':'siteTable'})
    OPtext = main_table.find("div", attrs={'class':'usertext-body'}).findAll('p')
    txt = ""
    for p in range(len(OPtext)):
        txt += OPtext[p].text

    OPwords = txt.split()


    OPname = main_table.find("a", attrs={'class':"author"})
    name = OPname.text
    
    return(txt, OPwords, name)

def getComments(url):
    deltabot = 0
    comment_area = souper(url).find('div',attrs={'class':'commentarea'})
    comments = comment_area.find_all('div', attrs={'class':'entry unvoted'})
    extracted_comments = []
    textComm = []
    names = []
    delta_link = []
    for comment in comments: 
        if comment.find('form'):
            link = comment.find('a',attrs={'class':'bylink'})['href']
            commenter = comment.find('a',attrs={'class':'author'}).text
            comment_text = comment.find('div',attrs={'class':'md'}).text
            extracted_comments.append({'name':commenter,'text':comment_text})
            textComm.append(comment_text)
            names.append(commenter)
        if commenter == "DeltaBot":
            deltabot = 1
            delta_link.append(link)
    return(extracted_comments, textComm, names, delta_link, deltabot)


def search(query):
    query = query.lower()
    TOPIC = None
    stop = False
    locURL = None
    deltabot = None
    url = "https://old.reddit.com/r/changemyview/"
    ##two vectors of topics and urls
    topics, urls = linksAndTopics(url)
    for i in range(len(topics)):
        splitTOP = topics[i].split()
        for x in range(len(splitTOP)):
            word = splitTOP[x].lower()
            if word == query:
                TOPIC = topics[i]
                #print(i)
                locURL = i
                stop = True
                break
    if stop == False:
        return(None,None,None,None,None, "No topic found")
    else:
        ##return all data
        URL = urls[locURL]
        OPtxt, OPwords, Opname = getOPtext(URL)
        comments, commentTXT, Names, deltalink, deltabot = getComments(URL)
        return(OPname, OPtxt, commentTXT, Names, URL, TOPIC, deltalink, deltabot)   
    

OPname, OPtxt, CommentText, commentName, URL, TOPIC, deltalink, deltabot= search("unreasonable")

print(TOPIC)
print(URL)
print(OPtxt)
print(CommentText[0])


CMV: It is unreasonable to expect consumers to wait for international releases of media, and piracy is basically justified in these instances.
https://old.reddit.com/r/changemyview/comments/9om3j8/cmv_it_is_unreasonable_to_expect_consumers_to/
So for example if a TV show releases in America first, but then it takes months for that show to be released in another country, then you shouldn't be surprised if tons of people resort to pirating your show instead. We live in an age of instant gratification and this is just how the world works now, if you aren't prepared to give audiences worldwide a way to pay for and access your content legally and conveniently then that's your problem.I think services like Netflix exemplify this problem - you are offered different content depending on where you are. In that case you can't even use the 'lost sale' argument (which is already a tenuous claim) because you're not paying the creators of the show directly, you're paying Netflix, and if you're alrea

In [264]:
deltalink

['https://old.reddit.com/r/changemyview/comments/9om3j8/cmv_it_is_unreasonable_to_expect_consumers_to/e7v3q2u/',
 'https://old.reddit.com/r/changemyview/comments/9om3j8/cmv_it_is_unreasonable_to_expect_consumers_to/e7v3q3l/',
 'https://old.reddit.com/r/changemyview/comments/9om3j8/cmv_it_is_unreasonable_to_expect_consumers_to/e7v4dk1/']

In [265]:
if deltabot == 1:
    deltaTXT = CommentText[0]
    deltaW = deltaTXT.split()
    deltaNumber = int(deltaTXT.split()[4])
    print(deltaW)
    print(deltaNumber)
    print(deltalink[0])

from bs4 import BeautifulSoup

url = deltalink[0]
store = []

comment_area = souper(url).find('div',attrs={'class':'commentarea'})
comments = comment_area.find_all('div', attrs={'class':'entry unvoted'})
for comment in comments: 
        if comment.find('form'):
            delta = comment.find('div', {'class': 'md'})
            if delta.find('a', {'href': True}, text='here')!= None:
                store.append(delta.find('a', {'href': True}, text='here'))

deltaurl = str(store[0])
print(deltaurl)
deltaurl = "https://old.reddit.com"+ deltaurl[9:-25]

print(deltaurl)
#print(store)

store = []
main_table = souper(deltaurl).find("div",attrs={'id':'siteTable'})
entries = main_table.find_all("div",attrs={'id':'entry unvoted'})
for entry in entries:
    if entry.find('form'):
        delta = entry.find('div', {'class': 'md'})
        all = delta.find_all('a', {'href': True})
        
deltanames = []
for i in range(4,4+(deltaNumber*2),2):
    deltanames.append((all[i].text))


['/u/silenthunt', '(OP)', 'has', 'awarded', '1', 'delta(s)', 'in', 'this', 'post.', 'All', 'comments', 'that', 'earned', 'deltas', '(from', 'OP', 'or', 'other', 'users)', 'are', 'listed', 'here,', 'in', '/r/DeltaLog.', 'Please', 'note', 'that', 'a', 'change', 'of', 'view', "doesn't", 'necessarily', 'mean', 'a', 'reversal,', 'or', 'that', 'the', 'conversation', 'has', 'ended.', 'Delta', 'System', 'Explained', '|', 'Deltaboards']
1
https://old.reddit.com/r/changemyview/comments/9om3j8/cmv_it_is_unreasonable_to_expect_consumers_to/e7v3q2u/
<a href="/r/DeltaLog/comments/9omsby/deltas_awarded_in_cmv_it_is_unreasonable_to/" rel="nofollow">here</a>
https://old.reddit.com/r/DeltaLog/comments/9omsby/deltas_awarded_in_cmv_it_is_unreasonable_to/


In [266]:
print(deltanames)

['/u/TomorrowsBreakfast']
